In [ ]:
import argparse
import requests
import datetime
import os, errno
import logging
from subprocess import call
import shutil


In [ ]:
DIR = '../myopenaps'
NIGHTSCOUT_HOST = 'https://ns-drop-gd.herokuapp.com'
START_DATE = datetime.datetime.today() - datetime.timedelta(days=17)
END_DATE = datetime.datetime.today()
NUMBER_OF_RUNS = 1
EXPORT_EXCEL = None
TERMINAL_LOGGING = True
RECOMMENDS_REPORT = True
breakpoint()

In [ ]:
def get_input_arguments():
    parser = argparse.ArgumentParser(description='Autotune')
    
    # Required
    # NOTE: As the code runs right now, this directory needs to exist and as well as the subfolders: autotune, settings
    parser.add_argument('--dir',
                        '-d',
                        type=str,
                        required=False,
                        help='(--dir=<OpenAPS Directory>)')        
    parser.add_argument('--ns-host',
                        '-n',
                        type=str,
                        required=False,
                        metavar='NIGHTSCOUT_HOST',
                        help='(--ns-host=<NIGHTSCOUT SITE URL)')
    parser.add_argument('--start-date',
                        '-s',
                        type=lambda d: datetime.datetime.strptime(d, '%Y-%m-%d'),
                        required=False,
                        help='(--start-date=<YYYY-MM-DD>)')
    # Optional
    parser.add_argument('--end-date',
                        '-e',
                        type=lambda d: datetime.datetime.strptime(d, '%Y-%m-%d'),
                        help='(--end-date=<YYYY-MM-DD>) ')
    parser.add_argument('--runs',
                        '-r',
                        type=int,
                        metavar='NUMBER_OF_RUNS',
                        help='(--runs=<integer, number of runs desired>)')
    parser.add_argument('--xlsx',
                        '-x',
                        type=str,
                        metavar='EXPORT_EXCEL',
                        help='(--xlsx=<filenameofexcel>)')                
    parser.add_argument('--log',
                        '-l',
                        type=str,
                        metavar='TERMINAL_LOGGING',
                        help='(--log <true/false(true)>)')
    
    return parser.parse_args()


In [ ]:
%%debug

def assign_args_to_variables(args):
    # TODO: Input checking.
    
    global DIR, NIGHTSCOUT_HOST, START_DATE, END_DATE, NUMBER_OF_RUNS, \
           EXPORT_EXCEL, TERMINAL_LOGGING, RECOMMENDS_REPORT
    
    # On Unix and Windows, return the argument with an initial component of
    # ~ or ~user replaced by that user's home directory.
    # DIR = os.path.expanduser(args.dir)
    
    # NIGHTSCOUT_HOST = args.ns_host

    # START_DATE = args.start_date
    
    if args.end_date is not None:
        END_DATE = args.end_date
        
    if args.runs is not None:
        NUMBER_OF_RUNS = args.runs
        
    if args.xlsx is not None:
        EXPORT_EXCEL = args.xlsx
    
    if args.log is not None:
        RECOMMENDS_REPORT = args.logsa

In [ ]:
def get_nightscout_profile(nightscout_host):
    #TODO: Add ability to use API secret for Nightscout.
    autotune_directory = os.path.join(DIR, 'autotune')
    res = requests.get(nightscout_host + '/api/v1/profile.json')
    with open(os.path.join(autotune_directory, 'nightscout.profile.json'), 'w') as f:  # noqa: F821
        f.write(res.text)

In [ ]:
print(autotune_directory.text)

In [ ]:
def get_openaps_profile(directory):
    shutil.copy(os.path.join(directory, 'settings', 'pumpprofile.json'), os.path.join(directory, 'autotune', 'profile.pump.json'))
    
    # If a previous valid settings/autotune.json exists, use that; otherwise start from settings/profile.json
    
    # This allows manual users to be able to run autotune by simply creating a settings/pumpprofile.json file.
    # cp -up settings/pumpprofile.json settings/profile.json
    shutil.copy(os.path.join(directory, 'settings', 'pumpprofile.json'), os.path.join(directory, 'settings', 'profile.json'))
    shutil.copy(os.path.join(directory, 'settings', 'profile.json'), os.path.join(directory, 'settings', 'autotune.json'))
    
    # cp settings/autotune.json autotune/profile.json
    shutil.copy(os.path.join(directory, 'settings', 'autotune.json'), os.path.join(directory, 'autotune', 'profile.json'))
